In [ ]:
import random

In [ ]:
LINES = [
    (0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)
]

In [ ]:

def winner (board):
    for a,b,c in LINES:
        line =board[a] + board[b] + board[c]
        if line == "XXX" : return "X"
        if line =="000" :return "0"
    return None

In [ ]:
def empty_cells(board):
    return [i for i,v in enumerate(board) if v=="_"]

In [ ]:
def show(board,with_numbers=False):
    s="".join(str(i+1) if (with_numbers and board[i]=="_") else (board[i].replace("_","·"))
          for i in range(9))
    print(s[0],s[1], s[2])
    print(s[3],s[4], s[5])
    print(s[6],s[7], s[8])


In [ ]:
def step(state, a):
    if state[a] != "_":
       return state , -1.0, True
    b= state[:a] + "X" + state [a+1:]
    w =winner(b)
    if w =="X" : return b , +1.0, True
    if not empty_cells(b): return b ,-1.0, True

    op= random.choice(empty_cells(b))
    b= state[:op] + "O" + state [op+1:]
    w =winner(b)
    if w =="O": return b, -1.0 ,True
    if not empty_cells(b) : return b ,-1.0, True
    return b ,0.0, False


In [ ]:
Q={}
def get_q(state):
    if state not in Q:
        Q[state]=[0.0]*9
    return  Q[state]


In [ ]:
def best_action(state):
  q = get_q(state)[:]
  for i in range(9):
    if state[i] != "_":
      q[i] = -1e9
  return max(range(9), key=lambda i: q[i])

In [ ]:
def eps_action(state, eps=0.1):
    moves =[i for i in range(9) if state[i]=="_"]
    if not moves: return None
    return random.choice(moves) if random.random() < eps  else best_action(state)

In [ ]:
episodes =99000
eps=1.0 #100% exploration
eps_min=0.01
eps_decay=0.999
gamma=0.9
learning_rate=0.1

In [ ]:
for ep in range (1,episodes+1):
    s="_________"
    done=False
    while not done:
        a = eps_action(s,eps)
        s2,r,done =step(s,a)
        if done:
           target=r
        else :
           next_moves=  [i for i in range(9) if s2[i]=="_"]
           next_max= 0.0 if not next_moves else max(get_q((s2)[i] for i in next_moves))
           target = r + gamma + next_max
        q = get_q(s)
        q[a] += learning_rate * ( target -q[a])
        s=s2
        eps =max(eps_min, eps*eps_decay)
    print(f"reached epidose {ep}/{episodes} ")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
reached epidose 94001/99000 
reached epidose 94002/99000 
reached epidose 94003/99000 
reached epidose 94004/99000 
reached epidose 94005/99000 
reached epidose 94006/99000 
reached epidose 94007/99000 
reached epidose 94008/99000 
reached epidose 94009/99000 
reached epidose 94010/99000 
reached epidose 94011/99000 
reached epidose 94012/99000 
reached epidose 94013/99000 
reached epidose 94014/99000 
reached epidose 94015/99000 
reached epidose 94016/99000 
reached epidose 94017/99000 
reached epidose 94018/99000 
reached epidose 94019/99000 
reached epidose 94020/99000 
reached epidose 94021/99000 
reached epidose 94022/99000 
reached epidose 94023/99000 
reached epidose 94024/99000 
reached epidose 94025/99000 
reached epidose 94026/99000 
reached epidose 94027/99000 
reached epidose 94028/99000 
reached epidose 94029/99000 
reached epidose 94030/99000 
reached epidose 94031/99000 
reached epidose 94032/9

In [ ]:
def model_move(board):  # X plays
    s = board
    a = best_action(s)
    if s[a] != "_": return s  # safety
    return s[:a] + "X" + s[a+1:]

def human_move(board):  # O plays
    while True:
        try:
            raw = input("Pick a cell (1-9): ").strip()
            if not raw.isdigit(): raise ValueError
            a = int(raw) - 1
            if a not in range(9) or board[a] != "_": raise ValueError
            return board[:a] + "O" + board[a+1:]
        except ValueError:
            print("Invalid. Choose an empty cell number from the board.")

def play_once():
    b = "_________"
    turn = "X"  # model starts
    print("\nNew game! Empty board. Numbers show where you can play.")
    while True:
        print("\nBoard:")
        show(b, with_numbers=True)
        w = winner(b)
        if w or not empty_cells(b):
            print("\nFinal:")
            show(b)
            if   w == "X": print("🤖 Model (X) wins!")
            elif w == "O": print("🎉 You (O) win!")
            else:          print("🤝 Draw.")
            break
        if turn == "X":
            b = model_move(b)
            turn = "O"
        else:
            b = human_move(b)
            turn = "X"

# Play loop
while True:
    play_once()
    if input("Play again? (y/n): ").strip().lower() != "y":
        print("Bye!"); break




New game! Empty board. Numbers show where you can play.

Board:
1 2 3
4 5 6
7 8 9

Board:
1 2 3
4 X 6
7 8 9
Pick a cell (1-9): 8

Board:
1 2 3
4 X 6
7 O 9

Board:
X 2 3
4 X 6
7 O 9
Pick a cell (1-9): 9

Board:
X 2 3
4 X 6
7 O O

Board:
X X 3
4 X 6
7 O O
Pick a cell (1-9): 7

Board:
X X 3
4 X 6
O O O

Board:
X X X
4 X 6
O O O

Final:
X X X
· X ·
O O O
🤖 Model (X) wins!
Play again? (y/n): y

New game! Empty board. Numbers show where you can play.

Board:
1 2 3
4 5 6
7 8 9

Board:
1 2 3
4 X 6
7 8 9
Pick a cell (1-9): 1

Board:
O 2 3
4 X 6
7 8 9

Board:
O X 3
4 X 6
7 8 9
Pick a cell (1-9): 8

Board:
O X 3
4 X 6
7 O 9

Board:
O X X
4 X 6
7 O 9
Pick a cell (1-9): 9

Board:
O X X
4 X 6
7 O O

Board:
O X X
X X 6
7 O O
Pick a cell (1-9): 6

Board:
O X X
X X O
7 O O

Board:
O X X
X X O
X O O

Final:
O X X
X X O
X O O
🤖 Model (X) wins!
Play again? (y/n): y

New game! Empty board. Numbers show where you can play.

Board:
1 2 3
4 5 6
7 8 9

Board:
1 2 3
4 X 6
7 8 9
Pick a cell (1-9): 5
Invalid. Ch